In [1]:
#import statements, weight definitions, data classes, rounding function
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore') 

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from numpy import random




#Data class
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

    
    
    
    
#Data loaders (to look at data) and deal with data (training and testing)
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
    
    

def binary_acc(y_pred, y_test): 
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc




X=random.rand(1000, 5)

np.random.normal(0, 1, size=(1000, 10))


y=[]
for i in range(1000):
    y.append(random.binomial(1, 0.5, size=None))




In [2]:
#model class 5 layers, then an output layer
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.layer_1 = nn.Linear(5, 5) 
        self.layer_2 = nn.Linear(5, 5) 
        self.layer_3 = nn.Linear(5, 5)
        self.layer_4 = nn.Linear(5, 5)
        self.layer_5 = nn.Linear(5, 5) 
        self.layer_out = nn.Linear(5, 1) #output layer
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        
        self.batchnorm1 = nn.BatchNorm1d(5)
        self.batchnorm2 = nn.BatchNorm1d(5)
        self.batchnorm3 = nn.BatchNorm1d(5)
        self.batchnorm4 = nn.BatchNorm1d(5)
        self.batchnorm5 = nn.BatchNorm1d(5)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.batchnorm5(x)        
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [3]:
#parallel model (two 5 layer models, with output going into new output layer)
class parallelmodel(nn.Module):
    def __init__(self, originalmodel):
        super(parallelmodel, self).__init__()
        self.layer1_1 = copy.deepcopy(originalmodel.layer_1)
        self.layer1_2 = copy.deepcopy(originalmodel.layer_2)
        self.layer1_3 = copy.deepcopy(originalmodel.layer_3)
        self.layer1_4 = copy.deepcopy(originalmodel.layer_4)
        self.layer1_5 = copy.deepcopy(originalmodel.layer_5)
        self.layer1_out = copy.deepcopy(originalmodel.layer_out)
        
        self.relu1 = copy.deepcopy(originalmodel.relu)        
        self.dropout1 = copy.deepcopy(originalmodel.dropout)
        
        self.batchnorm1_1 = copy.deepcopy(originalmodel.batchnorm1)
        self.batchnorm1_2 = copy.deepcopy(originalmodel.batchnorm2)
        self.batchnorm1_3 = copy.deepcopy(originalmodel.batchnorm3)
        self.batchnorm1_4 = copy.deepcopy(originalmodel.batchnorm4)
        self.batchnorm1_5 = copy.deepcopy(originalmodel.batchnorm5)
        #self.batchnorm1_out=nn.BatchNorm1d(1)

        
        self.batchnorm2_1 = nn.BatchNorm1d(5)
        self.batchnorm2_2 = nn.BatchNorm1d(5)
        self.batchnorm2_3 = nn.BatchNorm1d(5)
        self.batchnorm2_4 = nn.BatchNorm1d(5)
        self.batchnorm2_5 = nn.BatchNorm1d(5)
        #self.batchnorm2_out=nn.BatchNorm1d()
        
        self.layer2_1 = nn.Linear(5, 5) 
        self.layer2_2 = nn.Linear(5, 5) 
        self.layer2_3 = nn.Linear(5, 5)
        self.layer2_4 = nn.Linear(5, 5)
        self.layer2_5 = nn.Linear(5, 5) 
        self.layer2_out = nn.Linear(5, 1) 
        
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.1)
        
        
        self.outputlayer= nn.Linear(2, 1)
        
        
        
        
        
        
    def forward(self, inputs):
        x = self.relu1(self.layer1_1(inputs))
        x = self.batchnorm1_1(x)
        x = self.relu1(self.layer1_2(x))
        x = self.batchnorm1_2(x)
        x = self.relu1(self.layer1_3(x))
        x = self.batchnorm1_3(x)
        x = self.relu1(self.layer1_4(x))
        x = self.batchnorm1_4(x)
        x = self.relu1(self.layer1_5(x))
        x = self.batchnorm1_5(x) 
        x = self.dropout1(x)
        x = self.layer1_out(x)
        x=torch.sigmoid(x)
        
        
        y = self.relu2(self.layer2_1(inputs))
        y = self.batchnorm2_1(y)
        y = self.relu2(self.layer2_2(y))
        y = self.batchnorm2_2(y)
        y = self.relu2(self.layer2_3(y))
        y = self.batchnorm2_3(y)
        y = self.relu2(self.layer2_4(y))
        y = self.batchnorm2_4(y)
        y = self.relu2(self.layer2_5(y))
        y = self.batchnorm2_5(y)        
        y = self.dropout2(y)
        y = self.layer2_out(y)
        y=torch.sigmoid(y)
        
        z=self.outputlayer(torch.cat([x, y], dim=1))
        return z
        
        
        

In [4]:
#groundtruth model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
scaler = StandardScaler()
X_allscaled = scaler.fit_transform(X) #scales data
X_formatted = testData(torch.FloatTensor(X_allscaled))
EPOCHS = 50 #number of passes of whole data
BATCH_SIZE = 64 #size of data going through at once
LEARNING_RATE = 0.001




Truthtrain=trainData(torch.FloatTensor(X_allscaled), 
                       torch.FloatTensor(y))


Truthloadertrain= DataLoader(dataset=Truthtrain, batch_size=BATCH_SIZE, shuffle=True)
GroundTruth = binaryClassification()


GroundTruth.to(device)
criterion = nn.BCEWithLogitsLoss()
truthoptimizer = optim.Adam(GroundTruth.parameters(), lr=LEARNING_RATE)

#print(list(model.parameters())[0])



baddata=True
while(baddata):

    

    GroundTruth.train()
    for e in range(1, EPOCHS+1):
        epoch_loss = 0
        epoch_acc = 0
        for X_batch, y_batch in Truthloadertrain:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            truthoptimizer.zero_grad()

            y_pred = GroundTruth(X_batch)

            loss = criterion(y_pred, y_batch.unsqueeze(1))
            acc = binary_acc(y_pred, y_batch.unsqueeze(1))

            loss.backward()
            truthoptimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()





    Truthtest = testData(torch.FloatTensor(X_allscaled))
    Truthloader = DataLoader(dataset=Truthtest, batch_size=1)



    truth_list = []
    GroundTruth.eval()
    with torch.no_grad():
        for X_batch in Truthloader:
            X_batch = X_batch.to(device)
            y_truth = GroundTruth(X_batch)
            y_truth = torch.sigmoid(y_truth)
            y_truthtag = torch.round(y_truth)
            truth_list.append(y_truthtag.cpu().numpy())

    y = [a.squeeze().tolist() for a in truth_list] #new truth values
    #split data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)
    #print(y_train)
    #rescale data 
    scaler = StandardScaler()
    truthcounter=0
    for i in y:
        if i==1:
            truthcounter=truthcounter+1
    
    
    #print(truthcounter)
    if truthcounter/len(y)<.8 and truthcounter/len(y)>.2:
        baddata=False

    




train_data= trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))



test_data = testData(torch.FloatTensor(X_test))

#data loader initiation
train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
test_loader = DataLoader(dataset=test_data, batch_size=1)




device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [5]:
#train first model
trained = binaryClassification()
trained.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(trained.parameters(), lr=LEARNING_RATE)

#w1=(trained.layer_1.weight)
#w2=(trained.layer_2.weight)
#w3=(trained.layer_3.weight)
#w4=(trained.layer_4.weight)
#w5=(trained.layer_5.weight)
#print(w1)
#print(w2)
#print(w3)
#print(w4)
#print(w5)
#print(trained.layer_out.weight)
#train trained model
trained.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = trained(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')


    
    
y_fullmodelpred_list =[]
trained.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = trained(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_fullmodelpred_list.append(y_pred_tag.cpu().numpy())


y_fullmodelpred_list = [a.squeeze().tolist() for a in y_fullmodelpred_list]



    
fullmodelcounter=0
for i in range(len(y_fullmodelpred_list)):
    if y_fullmodelpred_list[i]==y_test[i]:
        fullmodelcounter=fullmodelcounter+1       

print("# correct: ",fullmodelcounter, " out of ", len(y_test))
        
print("Full model statistics")
print(classification_report(y_test, y_fullmodelpred_list))    
    
    
        
    

# correct:  234  out of  330
Full model statistics
              precision    recall  f1-score   support

         0.0       0.69      0.71      0.70       157
         1.0       0.73      0.71      0.72       173

    accuracy                           0.71       330
   macro avg       0.71      0.71      0.71       330
weighted avg       0.71      0.71      0.71       330



In [6]:
#train parralel model
parralel = parallelmodel(trained)
parralel.to(device)
criterion = nn.BCEWithLogitsLoss()

parralel.outputlayer.weight=torch.nn.Parameter(data=torch.tensor([[.7,0.3]]), requires_grad=True)


paroptimizer = optim.Adam(parralel.parameters(), lr=LEARNING_RATE)



#EPOCHS = 300 #number of passes of whole data
#BATCH_SIZE = 16 #size of data going through at once



#train trained model
parralel.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        paroptimizer.zero_grad()
        
        y_pred = parralel(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        paroptimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')


    
    
y_parmodelpred_list =[]
parralel.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = parralel(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_parmodelpred_list.append(y_pred_tag.cpu().numpy())


y_parmodelpred_list = [a.squeeze().tolist() for a in y_parmodelpred_list]


#print(y_parmodelpred_list)
    
parmodelcounter=0
for i in range(len(y_fullmodelpred_list)):
    if y_parmodelpred_list[i]==y_test[i]:
        parmodelcounter=parmodelcounter+1       

print("# correct: ",parmodelcounter, " out of ", len(y_test))
        
print("Full model statistics")
print(classification_report(y_test, y_parmodelpred_list))    
    
    
print("output weight: ", parralel.outputlayer.weight)

# correct:  257  out of  330
Full model statistics
              precision    recall  f1-score   support

         0.0       0.88      0.62      0.73       157
         1.0       0.73      0.92      0.81       173

    accuracy                           0.78       330
   macro avg       0.80      0.77      0.77       330
weighted avg       0.80      0.78      0.77       330

output weight:  Parameter containing:
tensor([[1.1769, 0.6490]], requires_grad=True)


In [7]:
#print(w1)
#print(w2)
#print(w3)
#print(w4)
#print(w5)
#print(trained.layer_out.weight)


In [9]:
#half model stats

halfmodeltrained=binaryClassification()
halfmodellow=binaryClassification()

halfmodeltrained.layer_1 = copy.deepcopy(parralel.layer1_1)
halfmodeltrained.layer_2 = copy.deepcopy(parralel.layer1_2)
halfmodeltrained.layer_3 = copy.deepcopy(parralel.layer1_3)
halfmodeltrained.layer_4 = copy.deepcopy(parralel.layer1_4)
halfmodeltrained.layer_5 = copy.deepcopy(parralel.layer1_5)
halfmodeltrained.layer_out = copy.deepcopy(parralel.layer1_out)


halfmodellow.layer_1 = copy.deepcopy(parralel.layer2_1)
halfmodellow.layer_2 = copy.deepcopy(parralel.layer2_2)
halfmodellow.layer_3 = copy.deepcopy(parralel.layer2_3)
halfmodellow.layer_4 = copy.deepcopy(parralel.layer2_4)
halfmodellow.layer_5 = copy.deepcopy(parralel.layer2_5)
halfmodellow.layer_out = copy.deepcopy(parralel.layer2_out)


print("pretrained stats:")



y_pretrianedlist =[]
halfmodeltrained.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = halfmodeltrained(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pretrianedlist.append(y_pred_tag.cpu().numpy())


y_pretrianedlist = [a.squeeze().tolist() for a in y_pretrianedlist]



correctcounter=0
for i in range(len(y_pretrianedlist)):
    if y_pretrianedlist[i]==y_test[i]:
        correctcounter=correctcounter+1 
        
        
print("Tained half model;")
print("Number correct full model: ", correctcounter, " out of ", len(y_test))
print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")
print(classification_report(y_test, y_pretrianedlist))





print("untrained half stats:")
y_untrianedlist =[]
halfmodellow.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = halfmodellow(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_untrianedlist.append(y_pred_tag.cpu().numpy())


y_untrianedlist = [a.squeeze().tolist() for a in y_untrianedlist]



correctcounter=0
for i in range(len(y_untrianedlist)):
    if y_untrianedlist[i]==y_test[i]:
        correctcounter=correctcounter+1 
        
        
print("untained half model;")
print("Number correct full model: ", correctcounter, " out of ", len(y_test))
print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")
print(classification_report(y_test, y_untrianedlist))






pretrained stats:
Tained half model;
Number correct full model:  157  out of  330
precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
              precision    recall  f1-score   support

         0.0       0.48      1.00      0.64       157
         1.0       0.00      0.00      0.00       173

    accuracy                           0.48       330
   macro avg       0.24      0.50      0.32       330
weighted avg       0.23      0.48      0.31       330

untrained half stats:
untained half model;
Number correct full model:  157  out of  330
precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)
              precision    recall  f1-score   support

         0.0       0.48      1.00      0.64       157
         1.0       0.00      0.00      0.00       173

    accuracy                           0.48       330
   macro avg       0.24      0.50      0.32       330
weighted avg       0.23      0.48      0.31       330



In [10]:
#multiple routines
val = input("Enter number of training routines:")
for k in range(int(val)):




    X=random.rand(1000, 5)

    np.random.normal(0, 1, size=(1000, 10))


    y=[]
    for i in range(1000):
        y.append(random.binomial(1, 0.5, size=None))

    
    truth_list = []
    GroundTruth.eval()
    with torch.no_grad():
        for X_batch in Truthloader:
            X_batch = X_batch.to(device)
            y_truth = GroundTruth(X_batch)
            y_truth = torch.sigmoid(y_truth)
            y_truthtag = torch.round(y_truth)
            truth_list.append(y_truthtag.cpu().numpy())

    y = [a.squeeze().tolist() for a in truth_list] #new truth values
    #split data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)






    train_data= trainData(torch.FloatTensor(X_train), 
                           torch.FloatTensor(y_train))



    test_data = testData(torch.FloatTensor(X_test))

    #data loader initiation
    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
    test_loader = DataLoader(dataset=test_data, batch_size=1)





    #train trained model
    parralel.train()
    for e in range(1, EPOCHS+1):
        epoch_loss = 0
        epoch_acc = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            paroptimizer.zero_grad()

            y_pred = parralel(X_batch)

            loss = criterion(y_pred, y_batch.unsqueeze(1))
            acc = binary_acc(y_pred, y_batch.unsqueeze(1))

            loss.backward()
            paroptimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()


        #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')




    y_parmodelpred_list =[]
    parralel.eval()
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = parralel(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_pred_tag = torch.round(y_test_pred)
            y_parmodelpred_list.append(y_pred_tag.cpu().numpy())


    y_parmodelpred_list = [a.squeeze().tolist() for a in y_parmodelpred_list]


    #print(y_parmodelpred_list)

    parmodelcounter=0
    for i in range(len(y_fullmodelpred_list)):
        if y_parmodelpred_list[i]==y_test[i]:
            parmodelcounter=parmodelcounter+1       
    print("training routine #: ", k+1)
    print("# correct: ",parmodelcounter, " out of ", len(y_test))

    print("Full model statistics")
    print(classification_report(y_test, y_parmodelpred_list))    
    print("output weight: ", parralel.outputlayer.weight)
    


Enter number of training routines:2
training routine #:  1
# correct:  168  out of  330
Full model statistics
              precision    recall  f1-score   support

         0.0       0.48      0.31      0.38       157
         1.0       0.52      0.69      0.59       173

    accuracy                           0.51       330
   macro avg       0.50      0.50      0.49       330
weighted avg       0.50      0.51      0.49       330

output weight:  Parameter containing:
tensor([[0.9721, 0.6614]], requires_grad=True)
training routine #:  2
# correct:  155  out of  330
Full model statistics
              precision    recall  f1-score   support

         0.0       0.41      0.27      0.33       157
         1.0       0.50      0.65      0.56       173

    accuracy                           0.47       330
   macro avg       0.45      0.46      0.45       330
weighted avg       0.46      0.47      0.45       330

output weight:  Parameter containing:
tensor([[0.9296, 0.6703]], requires_gra